In [3]:
import numpy as np
import pandas as pd
import os, os.path
import sqlite3
import tempfile

# Wstęp

Praca stanowi zadanie zaliczeniowe nr 3 z przedmiotu "Przetwarzanie danych w językach R i Python" prowadzonego na MiNI PW przez dr Annę Cenę w semestrze letnim 2020/2021.

Wykonane zostały dwa zapytania dotyczące danych, oba z użyciem funkcji pandas oraz kwerendy sql. Przy pomocy funkcji equals porównano wyniki obu metod. 

Wykorzystane dane pochodzą z portalu stackexchange.com. Zostały zanonimizowane i uproszczone. 
Źródło danych: https://www.gagolewski.com/resources/data/travel_stackexchange_com. 

W celu umożliwienia wykonania kwerendy sql, utoworzono tymczasową bazę danych i zaaranżowano z nią połączenie.

## Wczytanie danych

In [4]:
# read the data
base_path = r"."
Posts = pd.read_csv(base_path + r"\travel_stackexchange_com/Posts.csv.gz", compression = 'gzip')
Users = pd.read_csv(base_path + r"\travel_stackexchange_com/Users.csv.gz", compression = 'gzip')
Votes = pd.read_csv(base_path + r"\travel_stackexchange_com/Votes.csv.gz", compression = 'gzip')

## Stworzenie połączenia z utworzoną bazą danych 

In [5]:
# arrange temp sql database 
baza = os.path.join(tempfile.mkdtemp(), 'przyklad.db')
if os.path.isfile(baza): 
    os.remove(baza) 

# load data
conn = sqlite3.connect(baza) 
Posts.to_sql("Posts", conn)
Users.to_sql("Users", conn)
Votes.to_sql("Votes", conn)


# Zadanie 1

## SQL

In [6]:
df_sql_1 = pd.read_sql_query("""
SELECT UpVotesTab.*, Posts.Title FROM
(
    SELECT PostId, COUNT(*) AS UpVotes
    FROM Votes
    WHERE VoteTypeId=2
    GROUP BY PostId
) AS UpVotesTab
JOIN Posts ON UpVotesTab.PostId=Posts.Id
WHERE Posts.PostTypeId=1
ORDER BY UpVotesTab.UpVotes DESC
LIMIT 10
""", conn)

df_sql_1

,PostId,UpVotes,Title
0,3080,307,"OK we're all adults here, so really, how on ea..."
1,38177,254,How do you know if Americans genuinely/literal...
2,24540,221,How to intentionally get denied entry to the U...
3,20207,211,Why are airline passengers asked to lift up wi...
4,96447,178,Why prohibit engine braking?
5,98367,176,"Boss is asking for passport, but it has a stam..."
6,1224,157,Tactics to avoid getting harassed by corrupt p...
7,2539,149,How to avoid drinking vodka?
8,70827,143,"OK, we are all adults here, so what is a bidet..."
9,66845,142,I don't know my nationality. How can I visit D...


## Pandas 

In [7]:
# create UpVotesTab
UpVotesTab= Votes.loc[Votes["VoteTypeId"]==2, "PostId"] \
    .value_counts() \
    .to_frame() \
    .rename(columns = {"PostId": "UpVotes"})

# mege dfs
df_pd_1 = UpVotesTab.merge(Posts[Posts["PostTypeId"]==1], left_index=True, right_on="Id")

# sort,limit and clean the df to get identical with sql
df_pd_1 = df_pd_1.sort_values(by = "UpVotes", ascending=False) \
    .head(10)[["Id", "UpVotes", "Title"]] \
    .rename(columns = {"Id" : "PostId"}) \
    .reset_index(drop = True)

df_pd_1 

,PostId,UpVotes,Title
0,3080,307,"OK we're all adults here, so really, how on ea..."
1,38177,254,How do you know if Americans genuinely/literal...
2,24540,221,How to intentionally get denied entry to the U...
3,20207,211,Why are airline passengers asked to lift up wi...
4,96447,178,Why prohibit engine braking?
5,98367,176,"Boss is asking for passport, but it has a stam..."
6,1224,157,Tactics to avoid getting harassed by corrupt p...
7,2539,149,How to avoid drinking vodka?
8,70827,143,"OK, we are all adults here, so what is a bidet..."
9,66845,142,I don't know my nationality. How can I visit D...


## Porównanie

In [8]:
df_pd_1.equals(df_sql_1)

True

# Zadanie 5

## SQL 

In [9]:
df_sql_5 = pd.read_sql_query("""
SELECT 
    Questions.Id,
    Questions.Title,
    BestAnswers.MaxScore,
    Posts.Score AS AcceptedScore,
    BestAnswers.MaxScore-Posts.Score AS Difference
FROM (
    SELECT Id, ParentId, MAX(Score) AS MaxScore
    FROM Posts
    WHERE PostTypeId==2
    GROUP BY ParentId
) AS BestAnswers
JOIN (
    SELECT * FROM Posts
    WHERE PostTypeId==1
) AS Questions
ON Questions.Id=BestAnswers.ParentId
JOIN Posts ON Questions.AcceptedAnswerId=Posts.Id
ORDER BY Difference DESC
LIMIT 10
""", conn)

df_sql_5

,Id,Title,MaxScore,AcceptedScore,Difference
0,99713,"In Germany, what are you supposed to do if you...",126,33,93
1,81376,What is way to eat rice with hands in front of...,120,30,90
2,76737,Why do many countries in the world still requi...,118,31,87
3,38177,How do you know if Americans genuinely/literal...,245,166,79
4,90636,Caught speeding 111 Mph (179 km/h) in Californ...,101,25,76
5,81492,Why is international first class much more exp...,90,21,69
6,94320,Strange looking region of France on Google Maps,71,7,64
7,80050,What's an easy way of making my luggage unique...,72,16,56
8,69224,How can a tourist obtain an 'authentic' duelli...,96,47,49
9,20171,How to prevent the passenger in front of you f...,59,12,47


In [10]:
# create BestAnswers and Questions dfs
BestAnswers = Posts.loc[Posts["PostTypeId"]==2, ["ParentId", "Score"]].groupby("ParentId").agg("max").rename(columns = {"Score":"MaxScore"})
Questions = Posts.loc[Posts["PostTypeId"]==1, ["Id", "AcceptedAnswerId", "Title"]]

# merge dataframes
df_pd_5 = Posts[["Score", "Id"]].merge(
    BestAnswers.merge(Questions, left_on="ParentId", right_on="Id"), 
    left_on="Id", 
    right_on="AcceptedAnswerId")

# add Difference column
df_pd_5["Difference"] = df_pd_5["MaxScore"] - df_pd_5["Score"]

# sort, limit and clean the df to get identical with sql
df_pd_5 = df_pd_5.sort_values(by = "Difference", ascending=False).head(10) \
    .drop(columns=["Id_x"]) \
    .reset_index(drop=True) \
    .rename(columns={"Id_y":"Id", "Score": "AcceptedScore"}) \
    [["Id", "Title", "MaxScore", "AcceptedScore", "Difference"]]

df_pd_5

,Id,Title,MaxScore,AcceptedScore,Difference
0,99713,"In Germany, what are you supposed to do if you...",126,33,93
1,81376,What is way to eat rice with hands in front of...,120,30,90
2,76737,Why do many countries in the world still requi...,118,31,87
3,38177,How do you know if Americans genuinely/literal...,245,166,79
4,90636,Caught speeding 111 Mph (179 km/h) in Californ...,101,25,76
5,81492,Why is international first class much more exp...,90,21,69
6,94320,Strange looking region of France on Google Maps,71,7,64
7,80050,What's an easy way of making my luggage unique...,72,16,56
8,69224,How can a tourist obtain an 'authentic' duelli...,96,47,49
9,20171,How to prevent the passenger in front of you f...,59,12,47


## Porównanie


In [11]:
df_pd_5.equals(df_sql_5)

True

In [12]:
# close sql db connection
conn.close()

# Podsumowanie

Biblioteka pandas daje bardzo łatwy i przejrzysty sposób tworzenia zapytań dotyczących informacji zgromadzonych w ramkach danych. Bardzo łatwo również uzyskać można wyniki idenyczne z kwerendami SQL, a często sam sposób zapisu jest w pandas nawet bardziej intuicyjny.  